In [ ]:
import pandas as pd
import numpy as np
import glob
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from tensorflow.contrib import rnn
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
#https://www.tensorflow.org/tutorials/sequences/recurrent
#https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
#Fall labels: FOL, FKL, BSC, SDL
files_path = [
    'MobiAct_Dataset_v2.0/Annotated Data/FOL/*.csv',
    'MobiAct_Dataset_v2.0/Annotated Data/FKL/*.csv',
    'MobiAct_Dataset_v2.0/Annotated Data/BSC/*.csv',
    'MobiAct_Dataset_v2.0/Annotated Data/SDL/*.csv']

i = 0;
l_files = []
for path in files_path:
    for filename in glob.glob(path):
        fdf = pd.read_csv(filename)
        fdf['Id'] = [i for k in range(fdf.shape[0])]
        l_files.append(fdf)
        i += 1
    

In [ ]:
df = pd.concat(l_files, axis=0)

In [ ]:
labels = set(df.label)
labels

In [ ]:
df['label_fac'] = pd.factorize(df['label'])[0].astype(np.uint16)

In [ ]:
df_copy = df

In [ ]:
N_TIME_STEPS = 100
N_FEATURES = 3
step = 20
segments = []
labels = []

for i in range(0, len(df) - N_TIME_STEPS, step):
    acc_xs = df['acc_x'].values[i: i + N_TIME_STEPS]
    acc_ys = df['acc_y'].values[i: i + N_TIME_STEPS]
    acc_zs = df['acc_z'].values[i: i + N_TIME_STEPS]
#     gyro_xs = df['gyro_x'].values[i: i + N_TIME_STEPS]
#     gyro_ys = df['gyro_y'].values[i: i + N_TIME_STEPS]
#     gyro_zs = df['gyro_z'].values[i: i + N_TIME_STEPS]
#     azimuth = df['azimuth'].values[i: i + N_TIME_STEPS]
#     pitch = df['gyro_z'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['label'][i: i + N_TIME_STEPS])[0][0]
    segments.append([acc_xs, acc_ys, acc_zs])
    labels.append(label)

In [ ]:
np.array(segments).shape

In [ ]:
reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

In [ ]:
reshaped_segments.shape

In [ ]:
labels[0]

In [ ]:
RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(
        reshaped_segments, labels, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
N_CLASSES = 6
N_HIDDEN_UNITS = 64

def create_LSTM_model(inputs):
    W = {
        'hidden': tf.Variable(tf.random_normal([N_FEATURES, N_HIDDEN_UNITS])),
        'output': tf.Variable(tf.random_normal([N_HIDDEN_UNITS, N_CLASSES]))
    }
    biases = {
        'hidden': tf.Variable(tf.random_normal([N_HIDDEN_UNITS], mean=1.0)),
        'output': tf.Variable(tf.random_normal([N_CLASSES]))
    }
    
    X = tf.transpose(inputs, [1, 0, 2])
    X = tf.reshape(X, [-1, N_FEATURES])
    hidden = tf.nn.relu(tf.matmul(X, W['hidden']) + biases['hidden'])
    hidden = tf.split(hidden, N_TIME_STEPS, 0)

    # Stack 2 LSTM layers
    lstm_layers = [tf.contrib.rnn.BasicLSTMCell(N_HIDDEN_UNITS, forget_bias=1.0) for _ in range(2)]
    lstm_layers = tf.contrib.rnn.MultiRNNCell(lstm_layers)

    outputs, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden, dtype=tf.float32)

    # Get output for the last time step
    lstm_last_output = outputs[-1]

    return tf.matmul(lstm_last_output, W['output']) + biases['output']

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, N_TIME_STEPS, N_FEATURES], name="input")
Y = tf.placeholder(tf.float32, [None, N_CLASSES])

In [ ]:
pred_Y = create_LSTM_model(X)

pred_softmax = tf.nn.softmax(pred_Y, name="y_")

In [ ]:
L2_LOSS = 0.0015

l2 = L2_LOSS * \
    sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred_Y, labels = Y)) + l2

In [ ]:
LEARNING_RATE = 0.0025

optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred_softmax, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

In [ ]:
N_EPOCHS = 50
BATCH_SIZE = 1024
saver = tf.train.Saver()

history = dict(train_loss=[], 
                     train_acc=[], 
                     test_loss=[], 
                     test_acc=[])

sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

train_count = len(X_train)

for i in range(1, N_EPOCHS + 1):
    for start, end in zip(range(0, train_count, BATCH_SIZE),
                          range(BATCH_SIZE, train_count + 1,BATCH_SIZE)):
        sess.run(optimizer, feed_dict={X: X_train[start:end],
                                       Y: y_train[start:end]})

    _, acc_train, loss_train = sess.run([pred_softmax, accuracy, loss], feed_dict={
                                            X: X_train, Y: y_train})

    _, acc_test, loss_test = sess.run([pred_softmax, accuracy, loss], feed_dict={
                                            X: X_test, Y: y_test})

    history['train_loss'].append(loss_train)
    history['train_acc'].append(acc_train)
    history['test_loss'].append(loss_test)
    history['test_acc'].append(acc_test)

    if i != 1 and i % 10 != 0:
        continue

    print(f'epoch: {i} test accuracy: {acc_test} loss: {loss_test}')
    
predictions, acc_final, loss_final = sess.run([pred_softmax, accuracy, loss], feed_dict={X: X_test, Y: y_test})

print()
print(f'final results: accuracy: {acc_final} loss: {loss_final}')

In [ ]:
pickle.dump(predictions, open("predictions.p", "wb"))
pickle.dump(history, open("history.p", "wb"))
tf.train.write_graph(sess.graph_def, '.', 'har.pbtxt')  
saver.save(sess, save_path = "har.ckpt")


In [ ]:
sess.close()

In [ ]:
history = pickle.load(open("history.p", "rb"))
predictions = pickle.load(open("predictions.p", "rb"))

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(np.array(history['train_loss']), "r--", label="Train loss")
plt.plot(np.array(history['train_acc']), "g--", label="Train accuracy")
plt.plot(np.array(history['test_loss']), "r-", label="Test loss")
plt.plot(np.array(history['test_acc']), "g-", label="Test accuracy")
plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.show()

In [ ]:
from tensorflow.python.tools import freeze_graph

MODEL_NAME = 'har'

input_graph_path = MODEL_NAME+'.pbtxt'
checkpoint_path = MODEL_NAME+'.ckpt'
restore_op_name = "srestore_all"
filename_tensor_name = "Const:0"
output_frozen_graph_name = 'frozen_'+MODEL_NAME+'.pb'

freeze_graph.freeze_graph(input_graph_path, input_saver="",
                          input_binary=False, input_checkpoint=checkpoint_path, 
                          output_node_names="y_", restore_op_name="restore_all",
                          filename_tensor_name="Const:0", 
                          output_graph=output_frozen_graph_name, clear_devices=True, initializer_nodes="")